In [2]:
!pip install SpeechRecognition
!pip install vaderSentiment
!pip install pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 46.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.2 MB/s eta 0:00:00


In [9]:
import gdown
import speech_recognition as sr
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from pydub import AudioSegment

# Step 1: Convert Audio to Text
def convert_audio_to_text(audio_file):
    recognizer = sr.Recognizer()

    audio = AudioSegment.from_file(audio_file)
    audio = audio.set_channels(1).set_frame_rate(16000)

    with sr.AudioFile(audio.export(format="wav")) as source:
        audio_data = recognizer.record(source)

    try:
        conversation_text = recognizer.recognize_google(audio_data)
        return conversation_text
    except sr.UnknownValueError:
        print("Google Speech Recognition could not understand audio")
        return None
    except sr.RequestError as e:
        print(f"Could not request results from Google Speech Recognition service; {e}")
        return None

# Step 2: Text Preprocessing (simplified)
def preprocess_text(text):
    sentences = text.split(".")
    return sentences

# Step 3: Sentiment Analysis
def analyze_sentiment(sentences):
    analyzer = SentimentIntensityAnalyzer()
    positive, negative, neutral = 0, 0, 0
    sentiment_trend = []

    for sentence in sentences:
        sentiment = analyzer.polarity_scores(sentence)
        sentiment_trend.append(sentiment['compound'])

        if sentiment['compound'] >= 0.05:
            positive += 1
        elif sentiment['compound'] <= -0.05:
            negative += 1
        else:
            neutral += 1

    return positive, negative, neutral, sentiment_trend

# Step 4: Keyword Detection
def detect_keywords(text):
    positive_keywords = ["thank you", "happy", "satisfied", "great"]
    negative_keywords = ["not happy", "disappointed", "frustrated", "angry"]

    positive_matches = any(keyword in text.lower() for keyword in positive_keywords)
    negative_matches = any(keyword in text.lower() for keyword in negative_keywords)

    return positive_matches, negative_matches

# Step 5: Percentage Calculation
def calculate_percentage(positive, negative, neutral):
    total = positive + negative + neutral
    if total == 0:
        return 0, 0, 0
    positive_percentage = (positive / total) * 100
    negative_percentage = (negative / total) * 100
    neutral_percentage = (neutral / total) * 100
    return positive_percentage, negative_percentage, neutral_percentage

# Step 6: Determine Customer Satisfaction
def assess_satisfaction(positive_percentage, negative_percentage, sentiment_trend, positive_keywords, negative_keywords):
    satisfaction = "Neutral"

    # Check keyword matches
    if positive_keywords and not negative_keywords:
        satisfaction = "Positive"
    elif negative_keywords and not positive_keywords:
        satisfaction = "Negative"

    # Check sentiment trends
    if sentiment_trend and (sentiment_trend[-1] > sentiment_trend[0]):
        satisfaction = "Positive"
    elif sentiment_trend and (sentiment_trend[-1] < sentiment_trend[0]):
        satisfaction = "Negative"

    # Use percentage analysis as fallback
    if satisfaction == "Neutral":
        if positive_percentage > negative_percentage:
            satisfaction = "Positive"
        elif negative_percentage > positive_percentage:
            satisfaction = "Negative"

    return satisfaction

# Main function to run all steps and classify the call
def main(audio_file):
    print("Converting audio to text...")
    conversation_text = convert_audio_to_text(audio_file)
    if not conversation_text:
        print("No text extracted from audio.")
        return

    print("Preprocessing text...")
    sentences = preprocess_text(conversation_text)

    print("Analyzing sentiment...")
    positive, negative, neutral, sentiment_trend = analyze_sentiment(sentences)

    print("Detecting keywords...")
    positive_keywords, negative_keywords = detect_keywords(conversation_text)

    print("Calculating percentages...")
    positive_percentage, negative_percentage, neutral_percentage = calculate_percentage(positive, negative, neutral)

    print(f"Positive Sentiment: {positive_percentage:.2f}%")
    print(f"Negative Sentiment: {negative_percentage:.2f}%")
    print(f"Neutral Sentiment: {neutral_percentage:.2f}%")

    print("Assessing customer satisfaction...")
    satisfaction = assess_satisfaction(positive_percentage, negative_percentage, sentiment_trend, positive_keywords, negative_keywords)

    print(f"The call is classified as {satisfaction} in terms of customer satisfaction.")

if __name__ == "__main__":
    # Download file from Google Drive (update the URL with your file's ID)
    url = "https://drive.google.com/uc?id=1siHFN6GdFIFs9oaqou3sZ__UFtaCq7i7"
    output = "AudioCall.mp3"
    gdown.download(url, output, quiet=False)

    # Use the downloaded file
    audio_file_path = "AudioCall.mp3"
    main(audio_file_path)


Downloading...
From: https://drive.google.com/uc?id=1siHFN6GdFIFs9oaqou3sZ__UFtaCq7i7
To: /content/AudioCall.mp3
100%|██████████| 3.75M/3.75M [00:00<00:00, 112MB/s]


Converting audio to text...
Preprocessing text...
Analyzing sentiment...
Detecting keywords...
Calculating percentages...
Positive Sentiment: 100.00%
Negative Sentiment: 0.00%
Neutral Sentiment: 0.00%
Assessing customer satisfaction...
The call is classified as Positive in terms of customer satisfaction.
